In [1]:
import torch
from vocgan_generator import Generator
import soundfile as sf

In [2]:
torch.__version__

'1.11.0'

In [3]:
def get_vocgan(generator_path):
    generator = Generator(80, 4,
          ratios=[4, 4, 2, 2, 2, 2], mult=256,
          out_band=1)
    generator_checkpoint = torch.load(generator_path, map_location=torch.device('cpu'))
    generator.load_state_dict(generator_checkpoint['model_g'])
    return generator

In [4]:
generator_path = 'checkpoints/kss_29de09d_4500.pt'

In [5]:
generator = get_vocgan(generator_path)
generator.eval()
#generator = generator.cuda()

In [6]:
import numpy as np
import torch

from hparams import defaults
from model import Tacotron2
from train import load_model
from text import text_to_sequence

In [7]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

In [8]:
hparams = Struct(**defaults)

In [11]:
checkpoint_path = "output_ver2/checkpoint_130000"
model = load_model(hparams, torch.device('cpu'))
model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu'))['state_dict'])
model.eval()
#model.to(torch.device('cuda'))

Tacotron2(
  (embedding): Embedding(80, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (line

In [12]:
text = "이거 진짜 잘 되는거 맞음?"
sequence = np.array(text_to_sequence(text, ['korean_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).long()

In [13]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


Warning! Reached max decoder steps


In [14]:
audio = generator.generate_audio(mel_outputs_postnet)

In [15]:
sf.write('sample_audio1.wav', audio, hparams.sampling_rate, 'PCM_24')